In [34]:
# Import required libraries and install any necessary packages
import spacy
import warnings
import random

from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [35]:
import sys
import os

# Add the path to the directory containing the attribute_cleaner module
sys.path.append(os.path.abspath("../transformer"))

# Now you can import text_cleaner
from attribute_cleaner.general_string_cleaner import text_cleaner

In [36]:
# reading the data in doccano jsonl output format
with open('data/merged_data.jsonl', 'r') as f:
    lines = list(f)

training_data: list = []

for line in lines:
    row = json.loads(line)
    if row['label']:
        training_data.append(  [ row["text"], { "entities": row["label"] } ] )

random.shuffle(training_data)

print(len(training_data))

237


In [37]:
training_data[0]

['"our og best seller is back and this time , with an update ! this year\'s iteration of mind=blown is a yeast inoculated natural . the folks at mooleh manay wanted consistency across the board with their naturals and honeys this year . using yeast , was the best way to ensure this . for this year\'s lot , ripe cherries were picked and put in a floatation tank as usual to remove immatures . the cherries that sank were then put in airtight food grade barrels and a yeast culture was added . fermentation was carefully monitored and after 64 hours the coffee was put out to dry slowly on raised beds in a polyhouse ( the new normal thanks to unseasonal harvest rains ) . all this care has resulted in a coffee that is fruity but also clean and insanely sweet . we find the acidity very raspberry like and the sweetness reminds us of vanilla . we cannot state enough about how sweet and drinkable this year\'s iteration of mind=blown is . the coffee is very versatile and is a delight across all bre

In [38]:
# Define a function to create spaCy DocBin objects from the annotated data
def get_spacy_doc(file, data):
  # Create a blank spaCy pipeline
  nlp = spacy.blank('en')
  db = DocBin()

  # Iterate through the data
  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    # Extract entities from the annotations
    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity:
        continue

      entity_indices = entity_indices + list(range(start, end))
      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        # Log errors for annotations that couldn't be processed
        err_data = str([start, end]) + "    " + str(text) + "\n"
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db

In [39]:
# Split the annotated data into training and testing sets
from sklearn.model_selection import train_test_split
train, test = train_test_split(training_data, test_size=0.05)

# Display the number of items in the training and testing sets
print(len(train), len(test))

# Open a file to log errors during annotation processing
file = open('train_file_try_5.txt','w')

# Create spaCy DocBin objects for training and testing data
db = get_spacy_doc(file, train)
db.to_disk('train_doccano_try_5.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_doccano_try_5.spacy')

# Close the error log file
file.close()

225 12


100%|██████████| 12/12 [00:00<00:00, 739.82it/s]


In [40]:
# configure spacy for custom NER model , using a base config file - https://spacy.io/usage/training#config

# !python -m spacy init fill-config base_config.cfg config.cfg

### Model Training

In [41]:
!python -m spacy train config.cfg  --output  output_try_5  --paths.train train_doccano_try_5.spacy  --paths.dev  test_doccano_try_5.spacy --gpu-id 0

ℹ Saving to output directory: output_try_5
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/Users/snehil/.dev_env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/snehil/.dev_env/lib/python3.12/site-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.aut

### Loading and trying the model with new data

In [33]:
# Import the spaCy library
import spacy

# Load the trained spaCy NER model from the specified path
nlp = spacy.load('output_try_3/model-best')

/Users/snehil/.dev_env/lib/python3.12/site-packages/spacy_transformers/layers/hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self._model.load_state_dict(torch

In [ ]:
text = """
Jasmine kissed cranberry - 100 % arabica that is sourced from Chikmagalur, grown at an altitude of approximately 4,100ft to 4,500ft. It gets its unique name through a process which include a strain of yeast used to create a carbon dioxide-rich environment during fermentation in stainless-steel fermenters. A remarkable level of complexity in the beans is created by this process, laying the groundwork for a wonderful cup of coffee. Steady drying on raised beds is the next important step, which lets the flavors gradually develop and intensify. After another 30 days of continuous stirring, the coffee undergo additional drying developing scent of jasmine, complex notes of raspberries, cranberries, and sparkling malic acidity with a lingering floral aftertaste 
"""

In [ ]:
# Process the extracted text using the loaded spaCy NER model
doc = nlp(text_cleaner(text))

# Iterate through the named entities (entities) recognized by the model
for ent in doc.ents:
  # Print the recognized text and its corresponding label
  print(ent.text, "  ->>>>  ", ent.label_)

/home/snehil/.dev_env/lib/python3.12/site-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


jasmine   ->>>>   TASTING NOTES
arabica   ->>>>   COFFEE TYPE
chikmagalur   ->>>>   LOCATION
4 , 100ft   ->>>>   ELEVATION
jasmine   ->>>>   TASTING NOTES
raspberries   ->>>>   TASTING NOTES
cranberries   ->>>>   TASTING NOTES
malic   ->>>>   ACIDITY
floral   ->>>>   COFFEE_PROPERTIES


In [ ]:
text ="""
Single origin Indian Coffee From Moganad Estate, Tamil Nadu which is a 100% Arabiaca coffee with 
an exquisite blend of balanced sweetness and brightness. A Medium Dark Roast coffee with 
flavor notes of Cocoa, Caramel and Nut which can also be enjoyed in a French press, moka pot, aeropress & espresso .
 It is washed processed, grown at an altitude of around 4430 ft

"""

In [ ]:
# Process the extracted text using the loaded spaCy NER model
doc = nlp(text_cleaner(text))

# Iterate through the named entities (entities) recognized by the model
for ent in doc.ents:
  # Print the recognized text and its corresponding label
  print(ent.text, "  ->>>>  ", ent.label_)

moganad   ->>>>   ESTATE
tamil nadu   ->>>>   LOCATION
arabiaca   ->>>>   COFFEE TYPE
medium dark   ->>>>   ROAST LEVEL
cocoa   ->>>>   TASTING NOTES
caramel   ->>>>   TASTING NOTES
nut   ->>>>   TASTING NOTES
washed   ->>>>   PROCESSING
4430 ft   ->>>>   ELEVATION
